In [1]:
# timeit
# importing timeit
import timeit
code_to_test =  """
# Student Name : Georg Stotzingen
# Cohort       : 1 Lombard

# Note: You are only allowed to submit ONE final model for this assignment.


################################################################################
# Import Packages
################################################################################


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random as rand
import statsmodels.formula.api as smf
import sklearn.linear_model
import pydotplus
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz       
from sklearn.externals.six import StringIO     
from IPython.display import Image              
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import make_scorer



################################################################################
# Load Data
################################################################################

# use this space to load the original dataset
# MAKE SURE TO SAVE THE ORIGINAL FILE AS original_df
# Example: original_df = pd.read_excel('Apprentice Chef Dataset.xlsx')

original_df = pd.read_excel('C:/Users/Georg/Desktop/Hult/MSBA/Machine Learning/Case/Apprentice_Chef_Dataset.xlsx')




################################################################################
# Feature Engineering and (optional) Dataset Standardization
################################################################################

# use this space for all of the feature engineering that is required for your
# final model

# if your final model requires dataset standardization, do this here as well

# making a copy and shorter name to protect original_df

df=original_df.copy()

# lower casing for easier working
df.columns = map(str.lower, df.columns)

# adjusting display options and setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 200)

# calculating if they bought any beverages.

# from the case description, max price per meal is $23
# so every order above must include drinks
# max non alcoholic drink price is $5
# so every order above $28 must include wine
# also, we will try to factor in reductions for weekly plans
# added safeguard for new customers without revenue

df['avg_spend'] = df['revenue']/df['total_meals_ordered']
avg_spend_change_hi = 23
df['beverages'] = 0
condition = df.loc[0:,'beverages'][df['avg_spend'] > avg_spend_change_hi]
condition_0 = df.loc[0:,'beverages'][df['avg_spend'] == 0]

df['beverages'].replace(to_replace = condition,
                                      value      = 1,
                                      inplace    = True)
df['beverages'].replace(to_replace = condition_0,
                                      value      = 0,
                                      inplace    = True)

avg_spend_change_alc = 28
df['alcohol'] = 0
condition = df.loc[0:,'alcohol'][df['avg_spend'] > avg_spend_change_alc]
condition_0 = df.loc[0:,'alcohol'][df['avg_spend'] == 0]

df['alcohol'].replace(to_replace = condition,
                                      value      = 1,
                                      inplace    = True)
df['alcohol'].replace(to_replace = condition_0,
                                      value      = 0,
                                      inplace    = True)

# maximum average spend should be $23+25=48
# but some averages are much higher. Investigate further
# total meals ordered seems to be really number of total orders

# adding complaints as a percentage of total meals ordered
df['perc_complaints'] = df['contacts_w_customer_service']/df['total_meals_ordered']

# adding tendency to try new meals
df['new_meal_tendency'] = df['unique_meals_purch']/df['total_meals_ordered']

# adding perc of meals ordered as part of a meal plan, assuming an average 4 meals per plan
df['perc_meal_plan'] = df['weekly_plan']*4/df['total_meals_ordered']

# adding cancelations percentage of total orders
# busy customers could have different cross sell habits
df['perc_cancelations_before_noon'] = df['cancellations_before_noon']/df['total_meals_ordered']
df['perc_cancelations_after_noon'] = df['cancellations_after_noon']/df['total_meals_ordered']

#adding total_meals_ordered *followed_recommendations_pct/100
#shows how many orders where completed as recommendations
df['recommended_meals'] = df['total_meals_ordered']*df['followed_recommendations_pct']/100

# adding clicks_per_time
# this could estimate the activity while being on the website
df['clicks_per_time'] = df['avg_clicks_per_visit']/df['avg_time_per_site_visit']

#adding contacts_w_customer_service - early_deliveries -late_deliveries
#complaints would often be about the timing of the food
#complaints when the food was on time suggests other issues
df['on_time_complaints'] = df['contacts_w_customer_service']-df['early_deliveries']-df['late_deliveries']

# adding unique_meals_purch/product_categories_viewed
# this could be a proxy for curiosity
df['cat_viewed_purchased'] = df['unique_meals_purch']/df['product_categories_viewed']

# adding avg_time_per_site_visit/product_categories_viewed
# could show how quickly people decide
df['time_per_category'] = df['avg_time_per_site_visit']/df['product_categories_viewed']

#adding revenue/avg_time_per_site_visit
df['revenue_per_time'] = df['revenue']/df['avg_time_per_site_visit']

#adding revenue/avg_clicks_per_visit
df['revenue_per_click'] = df['revenue']/df['avg_clicks_per_visit']

#adding total cancelations
df['total_cancelations'] = df['cancellations_after_noon']+df['cancellations_before_noon']

#adding total_photos_viewed/avg_time_per_site_visit
df['photos_per_time'] = df['total_photos_viewed']/df['avg_time_per_site_visit']

#adding avg_time_per_site_visit + avg_prep_vid_time
#could proxy engagement 
df['total_time'] = df['avg_time_per_site_visit']+df['avg_prep_vid_time']

#adding largest_order_size*total_meals_ordered
#since we are not sure if total_meals_ordered is meals or orders, this is the maximum meals ordered
df['max_potential_meals'] = df['largest_order_size']*df['total_meals_ordered']

#adding total_meals_ordered/largest_order_size
#if it is number of meals, this is the minimum amount of orders
df['min_potential_orders'] = df['total_meals_ordered']/df['largest_order_size']

#adding revenue + avg_spend*cancellations_before_noon + avg_spend*cancellations_after_noon*0.5
#adjusting revenue for cancelations
df['adjusted_revenue'] = df['revenue']+df['avg_spend']*df['cancellations_before_noon']+ df['avg_spend']*df['cancellations_after_noon']*0.5

#adding mobile_logins + pc_logins
df['total_logins'] = df['mobile_logins']+df['pc_logins']

#adding followed_recommendations_pct*median_meal_rating
#could proxy satisfaction with recommendations
df['recommendations_satisfaction'] = df['followed_recommendations_pct']*df['median_meal_rating']

#adding followed_recommendations_pct/contacts_w_customer_service
df['complaints_on_recommendations'] = df['followed_recommendations_pct']/df['contacts_w_customer_service']

# creating columns for email categorization

# Splitting emails to categorise them

# STEP 1: splitting personal emails

# placeholder list
placeholder_lst = []

# looping over each email address
for index, col in df.iterrows():
    
    # splitting email domain at '@'
    split_email = df.loc[index, 'email'].split(sep = '@')
    
    # appending placeholder_lst with the results
    placeholder_lst.append(split_email)
    

# converting placeholder_lst into a DataFrame 
email_df = pd.DataFrame(placeholder_lst)


# STEP 2: concatenating with original DataFrame

# renaming column to concatenate
email_df.columns = ['0' , 'email_domain']


# concatenating email_domain with df DataFrame
df = pd.concat([df, email_df['email_domain']],
                     axis = 1)

# aggreate emails

# email domain types
personal_email_domains = [  '@gmail.com',
                            '@yahoo.com',
                            '@protonmail.com']
professional_email_domains = ['@mmm.com',
                            '@amex.com',
                            '@apple.com',
                            '@boeing.com',
                            '@caterpillar.com',
                            '@chevron.com',
                            '@cisco.com',
                            '@cocacola.com',
                            '@disney.com',
                            '@dupont.com',
                            '@exxon.com',
                            '@ge.org',
                            '@goldmansacs.com',
                            '@homedepot.com',
                            '@ibm.com',
                            '@intel.com',
                            '@jnj.com',
                            '@jpmorgan.com',
                            '@mcdonalds.com',
                            '@merck.com',
                            '@microsoft.com',
                            '@nike.com',
                            '@pfizer.com',
                            '@pg.com',
                            '@travelers.com',
                            '@unitedtech.com',
                            '@unitedhealth.com',
                            '@verizon.com',
                            '@visa.com',
                            '@walmart.com']

junk_email_domains  = [     '@me.com',
                            '@aol.com',
                            '@hotmail.com',
                            '@live.com',
                            '@msn.com',
                            '@passport.com']


# placeholder list
placeholder_lst = []


# looping to group observations by domain type
for domain in df['email_domain']:
    
    if '@' + domain in personal_email_domains:
        placeholder_lst.append('personal')
        

    elif '@' + domain in professional_email_domains:
        placeholder_lst.append('professional')
        
        
    elif '@' + domain in junk_email_domains:
        placeholder_lst.append('junk')


    else:
            print('Unknown')


# concatenating with original DataFrame
df['domain_group'] = pd.Series(placeholder_lst)

# creating dummy variables for email domain groups
one_hot_domain_group = pd.get_dummies(df['domain_group'])

# joining codings together
df = df.join([one_hot_domain_group])

# adding column for where first and last names are the same

df['dif_names'] = 0
condition = df.loc[0:,'first_name'] == df.loc[0:,'family_name']

df['dif_names'].replace(to_replace = condition,
                                value      = 1,
                                inplace    = True)

df['same_names'] = 0
names_placeholder_lst=[]
for i in df['dif_names']:
    
    if i == 0:
        names_placeholder_lst.append(1)
    else:
        names_placeholder_lst.append(0)
        
df['same_names'] = pd.Series(names_placeholder_lst)
        
# print(df['same_names'].value_counts())

#creating a dummy for families of at least four members

family_list= ['Frey',
'Targaryen',
'Stark',
'Tyrell',
'Lannister',
'Hightower',
'Greyjoy',
'Florent',
'Vance',
'Botley',
'Royce',
'Rivers',
'Martell',
'Crakehall',
'Harlaw',
'Arryn',
'Baratheon',
'Bracken',
'Sand',
'Waynwood',
'Velaryon',
'Seaworth',
'Westerling',
'Flowers',
'Plumm',
'Redwyne',
'Karstark',
'Fossoway',
'Osgrey',
'Brax',
'Mormont',
'Swann',
'I Targaryen',
'Wylde',
'Paege',
'Drumm',
'Goodbrother',
'Tallhart',
'zo Loraq',
'Beesbury',
'Mallister',
'Manderly',
'Glover',
'Umber',
'Estermont',
'Swyft',
'Dayne',
'Darry',
'Caswell',
'Farwynd',
'Norcross',
'Redfort',
'Blackwood',
'Heddle',
'Flint',
'II Targaryen',
'Hollard',
'Crane',
'Whent',
'Ryswell',
'Oakheart',
'Tully',
'Haigh',
'Crabb',
'Hornwood',
'Ambrose',
'Smallwood',
'Pyke',
'Connington',
'Ironmaker',
'Cassel',
'Lothston',
'Tarly',
'Hunter',
'Strong',
'Kettleblack',
'Slynt',
'Hill',
'Stokeworth',
'Storm',
'Caron',
'Yronwood',
'Farring',
'Manwoody',
'Goodbrook',
'III Targaryen', # manual cleanup of impurities in the dataset
'IV Targaryen',  # manual cleanup of impurities in the dataset
'V Targaryen',   # manual cleanup of impurities in the dataset
'Corbray']

df['family'] = 0
family_placeholder_lst=[]
for name in df['family_name']:
    
    if name in family_list:
        family_placeholder_lst.append(1)
    else:
        family_placeholder_lst.append(0)
        
df['family'] = pd.Series(family_placeholder_lst)
        
#print(df['family'].value_counts())

#creating a dummy for unique family names
family1_list=['Frey',
'Targaryen',
'Stark',
'Tyrell',
'Lannister',
'Hightower',
'Greyjoy',
'Florent',
'Vance',
'Botley',
'Royce',
'Rivers',
'Martell',
'Crakehall',
'Harlaw',
'Arryn',
'Baratheon',
'Bracken',
'Sand',
'Waynwood',
'Velaryon',
'Seaworth',
'Westerling',
'Flowers',
'Plumm',
'Redwyne',
'Karstark',
'Fossoway',
'Osgrey',
'Brax',
'Mormont',
'Swann',
'I Targaryen',
'Wylde',
'Paege',
'Drumm',
'Goodbrother',
'Tallhart',
'zo Loraq',
'Beesbury',
'Mallister',
'Manderly',
'Glover',
'Umber',
'Estermont',
'Swyft',
'Dayne',
'Darry',
'Caswell',
'Farwynd',
'Norcross',
'Redfort',
'Blackwood',
'Heddle',
'Flint',
'II Targaryen',
'Hollard',
'Crane',
'Whent',
'Ryswell',
'Oakheart',
'Tully',
'Haigh',
'Crabb',
'Hornwood',
'Ambrose',
'Smallwood',
'Pyke',
'Connington',
'Ironmaker',
'Cassel',
'Lothston',
'Tarly',
'Hunter',
'Strong',
'Kettleblack',
'Slynt',
'Hill',
'Stokeworth',
'Storm',
'Caron',
'Yronwood',
'Farring',
'Manwoody',
'Goodbrook',
'Corbray',
'Stackspear',
'Bulwer',
'Spicer',
'Penrose',
'Waters',
'Lefford',
'Staunton',
'Vypren',
'Stone',
'Hoare',
'Payne',
'Kenning',
'Penny',
'Piper',
'Uller',
'Stout',
'Darklyn',
'Qorgyle',
'Terys',
'Brune',
'Costayne',
'Locke',
'Bolton',
'Mooton',
'Santagar',
'Rowan',
'Morrigen',
'Ashford',
'of Oldtown',
'Grafton',
'Reed',
'Marbrand',
'Hardyng',
'Sharp',
'Blackmont',
'Webber',
'Willum',
'Snow',
'Alyn',
'Toyne',
'Meadows',
'Dalt',
'Jordayne',
'Rosby',
'Poole',
'Cuy',
'Uffering',
'Boy',
'of Pennytree',
'Wells',
'Hewett',
'Mullendore',
'Rykker',
'Selmy',
'Henly',
'Greenfield',
'Prestayn',
'Hetherspoon',
'Dothare',
'zo Galare',
'Dustin',
'Tarth',
'Mo',
'Ryger',
'Thorne',
'Inchfield',
'Hunt',
'Prester',
'Dondarrion',
'Humble',
'Allyrion',
'Tollett',
'Dick',
'Jeyne',
'Lonmouth',
'Serry',
'Rolfe',
'Lydden',
'Bernarr',
'Clifton',
'Lorch',
'Clegane',
'Blacktyde',
'Shepherd',
'Fell',
'Lynderly',
'Vaith',
'Norrey',
'Cerwyn',
'Nayland',
'Blackfyre',
'Charlton',
'Otherys',
'Wythers',
'Blackberry',
'Codd',
'Peake',
'Belmore',
'Ben',
'Woodwright',
'Wull',
'Knight',
'Wynch',
'King']

df['unique_name'] = 0
family1_placeholder_lst=[]
for name in df['family_name']:
    
    if name in family1_list:
        family1_placeholder_lst.append(0)
    else:
        family1_placeholder_lst.append(1)
        
df['unique_name'] = pd.Series(family1_placeholder_lst)
        
#print(df['unique_name'].value_counts())

#creating a dummy for family names that appear exactly twice
# this will help to spot a potential trend change from 1 to 2

family2_list=[
'Alyn',
'Toyne',
'Meadows',
'Dalt',
'Jordayne',
'Rosby',
'Poole',
'Cuy',
'Uffering',
'Boy',
'of Pennytree',
'Wells',
'Hewett',
'Mullendore',
'Rykker',
'Selmy',
'Henly',
'Greenfield',
'Prestayn',
'Hetherspoon',
'Dothare',
'zo Galare',
'Dustin',
'Tarth',
'Mo',
'Ryger',
'Thorne',
'Inchfield',
'Hunt',
'Prester',
'Dondarrion',
'Humble',
'Allyrion',
'Tollett',
'Dick',
'Jeyne',
'Lonmouth',
'Serry',
'Rolfe',
'Lydden',
'Bernarr',
'Clifton',
'Lorch',
'Clegane',
'Blacktyde',
'Shepherd',
'Fell',
'Lynderly',
'Vaith',
'Norrey',
'Cerwyn',
'Nayland',
'Blackfyre',
'Charlton',
'Otherys',
'Wythers',
'Blackberry',
'Codd',
'Peake',
'Belmore',
'Ben',
'Woodwright',
'Wull',
'Knight',
'Wynch',
'King']

df['name_twice'] = 0
family2_placeholder_lst=[]
for name in df['family_name']:
    
    if name in family2_list:
        family2_placeholder_lst.append(1)
    else:
        family2_placeholder_lst.append(0)
        
df['name_twice'] = pd.Series(family2_placeholder_lst)
        
#print(df['name_twice'].value_counts())

#setting outlier thresholds

revenue_hi                          = 5500
revenue_lo                          = 1285 #from looking at quantiles
total_meals_ordered_hi              = 220
total_meals_ordered_lo              = 16
unique_meals_purch_hi               = 8
contacts_w_customer_service_hi      = 11
contacts_w_customer_service_lo      = 3
product_categories_viewed_hi        = 10
product_categories_viewed_lo        = 2
avg_time_per_site_visit_hi          = 240
avg_time_per_site_visit_lo          = 20
cancellations_before_noon_hi        = 5
cancellations_after_noon_hi         = 1.5
mobile_logins_lo                    = 5
pc_logins_hi                        = 2
pc_logins_lo                        = 1
weekly_plan_hi                      = 20
early_deliveries_hi                 = 5
late_deliveries_hi                  = 7
avg_prep_vid_time_hi                = 250
largest_order_size_lo               = 2
largest_order_size_hi               = 7
master_classes_attended_hi          = 1
median_meal_rating_hi               = 3
median_meal_rating_lo               = 3
avg_clicks_per_visit_lo             = 8
avg_clicks_per_visit_hi             = 17
total_photos_viewed_hi              = 350
avg_spend_hi                        = 80
perc_complaints_hi                  = 0.27
new_meal_tendency_hi                = 0.3
perc_meal_plan_hi                   = 3

# Developing feature columns for outliers

# total_meals_ordered_hi_lo
df['out_total_meals_ordered'] = 0
condition_hi = df.loc[0:,'out_total_meals_ordered'][df['total_meals_ordered'] > total_meals_ordered_hi]
condition_lo = df.loc[0:,'out_total_meals_ordered'][df['total_meals_ordered'] < total_meals_ordered_lo]

df['out_total_meals_ordered'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)
df['out_total_meals_ordered'].replace(to_replace = condition_lo,
                                value      = 1,
                                inplace    = True)

# unique_meals_purch_hi
df['out_unique_meals_purch'] = 0
condition_hi = df.loc[0:,'out_unique_meals_purch'][df['unique_meals_purch'] > unique_meals_purch_hi]

df['out_unique_meals_purch'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# contacts_w_customer_service_hi_lo
df['out_contacts_w_customer_service'] = 0
condition_hi = df.loc[0:,'out_contacts_w_customer_service'][df['contacts_w_customer_service'] > contacts_w_customer_service_hi]
condition_lo = df.loc[0:,'out_contacts_w_customer_service'][df['contacts_w_customer_service'] < contacts_w_customer_service_lo]

df['out_contacts_w_customer_service'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

df['out_contacts_w_customer_service'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)

# product_categories_viewed_hi_lo
df['out_product_categories_viewed'] = 0
condition_hi = df.loc[0:,'out_product_categories_viewed'][df['product_categories_viewed'] > product_categories_viewed_hi]
condition_lo = df.loc[0:,'out_product_categories_viewed'][df['product_categories_viewed'] < product_categories_viewed_lo]

df['out_product_categories_viewed'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

df['out_product_categories_viewed'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)

# avg_time_per_site_visit_hi_lo
df['out_avg_time_per_site_visit'] = 0
condition_hi = df.loc[0:,'out_avg_time_per_site_visit'][df['avg_time_per_site_visit'] > avg_time_per_site_visit_hi]
condition_lo = df.loc[0:,'out_avg_time_per_site_visit'][df['avg_time_per_site_visit'] < avg_time_per_site_visit_lo]

df['out_avg_time_per_site_visit'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)
df['out_avg_time_per_site_visit'].replace(to_replace = condition_lo,
                                value      = 1,
                                inplace    = True)

# cancellations_before_noon_hi
df['out_cancellations_before_noon'] = 0
condition_hi = df.loc[0:,'out_cancellations_before_noon'][df['cancellations_before_noon'] > cancellations_before_noon_hi]

df['out_cancellations_before_noon'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# cancellations_after_noon_hi
df['out_cancellations_after_noon'] = 0
condition_hi = df.loc[0:,'out_cancellations_after_noon'][df['cancellations_after_noon'] > cancellations_after_noon_hi]

df['out_cancellations_after_noon'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# mobile_logins_lo
df['out_mobile_logins'] = 0
condition_lo = df.loc[0:,'out_mobile_logins'][df['mobile_logins'] < mobile_logins_lo]

df['out_mobile_logins'].replace(to_replace = condition_lo,
                                value      = 1,
                                inplace    = True)

# pc_logins_hi_lo
df['out_pc_logins'] = 0
condition_hi = df.loc[0:,'out_pc_logins'][df['pc_logins'] > pc_logins_hi]
condition_lo = df.loc[0:,'out_pc_logins'][df['pc_logins'] < pc_logins_lo]

df['out_pc_logins'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)
df['out_pc_logins'].replace(to_replace = condition_lo,
                                value      = 1,
                                inplace    = True)

# weekly_plan_hi
df['out_weekly_plan'] = 0
condition_hi = df.loc[0:,'out_weekly_plan'][df['weekly_plan'] > weekly_plan_hi]

df['out_weekly_plan'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# early_deliveries_hi
df['out_early_deliveries'] = 0
condition_hi = df.loc[0:,'out_early_deliveries'][df['early_deliveries'] > early_deliveries_hi]

df['out_early_deliveries'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# late_deliveries_hi
df['out_late_deliveries'] = 0
condition_hi = df.loc[0:,'out_late_deliveries'][df['late_deliveries'] > late_deliveries_hi]

df['out_late_deliveries'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# avg_prep_vid_time_hi
df['out_avg_prep_vid_time'] = 0
condition_hi = df.loc[0:,'out_avg_prep_vid_time'][df['avg_prep_vid_time'] > avg_prep_vid_time_hi]

df['out_avg_prep_vid_time'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# largest_order_size_hi_lo
df['out_largest_order_size'] = 0
condition_hi = df.loc[0:,'out_largest_order_size'][df['largest_order_size'] > largest_order_size_hi]
condition_lo = df.loc[0:,'out_largest_order_size'][df['largest_order_size'] < largest_order_size_lo]

df['out_largest_order_size'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

df['out_largest_order_size'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)

# master_classes_attended_hi
df['out_master_classes_attended'] = 0
condition_hi = df.loc[0:,'out_master_classes_attended'][df['master_classes_attended'] > master_classes_attended_hi]

df['out_master_classes_attended'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# median_meal_rating_hi
df['out_hi_median_meal_rating'] = 0
condition_hi = df.loc[0:,'out_hi_median_meal_rating'][df['median_meal_rating'] > median_meal_rating_hi]

df['out_hi_median_meal_rating'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)
# median_meal_rating_lo
df['out_lo_median_meal_rating'] = 0
condition_lo = df.loc[0:,'out_lo_median_meal_rating'][df['median_meal_rating'] < median_meal_rating_lo]

df['out_lo_median_meal_rating'].replace(to_replace = condition_lo,
                                value      = 1,
                                inplace    = True)


# avg_clicks_per_visit_lo_hi
df['out_avg_clicks_per_visit'] = 0
condition_hi = df.loc[0:,'out_avg_clicks_per_visit'][df['avg_clicks_per_visit'] > avg_clicks_per_visit_hi]
condition_lo = df.loc[0:,'out_avg_clicks_per_visit'][df['avg_clicks_per_visit'] < avg_clicks_per_visit_lo]

df['out_avg_clicks_per_visit'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)
df['out_avg_clicks_per_visit'].replace(to_replace = condition_lo,
                                value      = 1,
                                inplace    = True)

# total_photos_viewed_hi
df['out_total_photos_viewed'] = 0
condition_hi = df.loc[0:,'out_total_photos_viewed'][df['total_photos_viewed'] > total_photos_viewed_hi]

df['out_total_photos_viewed'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# revenue_hi
df['out_revenue'] = 0
condition_hi = df.loc[0:,'out_revenue'][df['revenue'] > revenue_hi]

df['out_revenue'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# lo_revenue_lo
df['out_lo_revenue'] = 0
condition_lo = df.loc[0:,'out_lo_revenue'][df['revenue'] < revenue_lo]

df['out_lo_revenue'].replace(to_replace = condition_lo,
                                value      = 1,
                                inplace    = True)

# avg_spend_hi
df['out_avg_spend'] = 0
condition_hi = df.loc[0:,'out_avg_spend'][df['avg_spend'] > avg_spend_hi]

df['out_avg_spend'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

#perc_complaints_hi
df['out_perc_complaints'] = 0
condition_hi = df.loc[0:,'out_perc_complaints'][df['perc_complaints'] > perc_complaints_hi]

df['out_perc_complaints'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

#new_meal_tendency_hi
df['out_new_meal_tendency'] = 0
condition_hi = df.loc[0:,'out_new_meal_tendency'][df['new_meal_tendency'] > new_meal_tendency_hi]

df['out_new_meal_tendency'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

#perc_meal_plan_hi
df['out_perc_meal_plan'] = 0
condition_hi = df.loc[0:,'out_perc_meal_plan'][df['perc_meal_plan'] > perc_meal_plan_hi]

df['out_perc_meal_plan'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


# declaring explanatory variables
df_data = df.drop(['cross_sell_success',
                   'name',
                   'email',
                   'first_name',
                   'family_name',
                   'email_domain',
                   'domain_group',
                   'junk',
                   'same_names'], 
                  axis = 1)

# declaring response variable
df_target = df.loc[ : , 'cross_sell_success']


candidate_dict = {
# lean set for Random forest
 'RF_lean'      : ['professional', 'name_twice',
                   'dif_names','followed_recommendations_pct'],
}

# preparing an empty list for scoring of the models
model_performance = [['Model', 'Training Accuracy',
                      'Testing Accuracy', 'AUC Value']]



################################################################################
# Train/Test Split
################################################################################

# use this space to set up testing and validation sets using train/test split

# Note: Be sure to set test_size = 0.25

# defining variable set
df_data   =  df.loc[ : , candidate_dict['RF_lean']]
df_target =  df.loc[ : , 'cross_sell_success']


# train/test split 
X_train, X_test, y_train, y_test = train_test_split(
            df_data,
            df_target,
            random_state = 222,
            test_size    = 0.25,
            stratify     = df_target)



################################################################################
# Final Model (instantiate, fit, and predict)
################################################################################

# use this space to instantiate, fit, and predict on your final model

# INSTANTIATING a random forest model with default values
rf_default = RandomForestClassifier(n_estimators     = 100,
                                    criterion        = 'gini',
                                    max_depth        = 8,
                                    min_samples_leaf = 1,
                                    bootstrap        = True,
                                    warm_start       = True,
                                    random_state     = 222)

# FITTING the training data
rf_default_fit = rf_default.fit(X_train, y_train)


# PREDICTING based on the testing set
rf_default_fit_pred = rf_default_fit.predict(X_test)


# SCORING the results
print('Training ACCURACY:', rf_default_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', rf_default_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = rf_default_fit_pred).round(4))




################################################################################
# Final Model Score (score)
################################################################################

# use this space to score your final model on the testing set
# MAKE SURE TO SAVE YOUR TEST SCORE AS test_score
# Example: test_score = final_model.score(X_test, y_test)

test_score = rf_default_fit.score(X_test, y_test).round(4)
auc_score = roc_auc_score(y_true  = y_test,
                                          y_score = rf_default_fit_pred).round(4)
                                          

"""

 

# calculating execution time
elapsed_time = timeit.timeit(code_to_test, number=3)/3
print(elapsed_time)




C:\Users\Georg\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


Training ACCURACY: 0.8005
Testing  ACCURACY: 0.8111
AUC Score        : 0.8187
Training ACCURACY: 0.8005
Testing  ACCURACY: 0.8111
AUC Score        : 0.8187
Training ACCURACY: 0.8005
Testing  ACCURACY: 0.8111
AUC Score        : 0.8187
12.324253200000001
